<a href="https://colab.research.google.com/github/kooll/ThinkPythonJ/blob/main/chapters/chap13_translated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Think Python 3e*の印刷版および電子書籍版は、[Bookshop.org](https://bookshop.org/a/98697/9781098155438)および[Amazon](https://www.amazon.com/_/dp/1098155432?smid=ATVPDKIKX0DER&_encoding=UTF8&tag=oreilly20-20&_encoding=UTF8&tag=greenteapre01-20&linkCode=ur2&linkId=e2a529f94920295d27ec8a06e757dc7c&camp=1789&creative=9325)から注文できます。

In [ ]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + str(local))
    return filename

download('https://github.com/AllenDowney/ThinkPython/raw/v3/thinkpython.py');
download('https://github.com/AllenDowney/ThinkPython/raw/v3/diagram.py');

import thinkpython

クレジット: 写真は[Lorem Picsum](https://picsum.photos/)からダウンロードしました。このサービスはプレースホルダー画像を提供しています。名前は、プレースホルダーテキストとして使われる "lorem ipsum" にちなんでいます。

In [ ]:
# This cell downloads an archive file that contains the the files we'll
# use for the examples in this chapter.

download('https://github.com/AllenDowney/ThinkPython/raw/v3/photos.zip');

In [ ]:
# WARNING: This cell removes the photos/ directory if it already exists.
# Any files already in the photos/ directory will be deleted.

# !rm -rf photos/

In [ ]:
!unzip -o photos.zip

# ファイルとデータベース

これまでに見てきたプログラムのほとんどは、実行時間が短く出力を生成するだけで、終了するとそのデータは消えてしまうという意味で、**一時的** なものでした。一時的なプログラムは実行するたびにクリーンな状態からスタートします。

一方で、他のプログラムは **持続的** です。これらは長時間（または常時）実行され、データの少なくとも一部を長期保存し、シャットダウンして再起動しても、停止したところから作業を再開できます。

プログラムがデータを保持する簡単な方法は、テキストファイルを読み書きすることです。もっと多機能な代替手段としては、データをデータベースに保存することです。データベースは、テキストファイルよりも効率的に読み書きができ、追加の機能を提供する特化したファイルです。

この章では、テキストファイルやデータベースを読み書きするプログラムを書きます。また、演習として、写真のコレクションから重複を検索するプログラムを書いてみましょう。でも、ファイルを扱う前に、まずはファイル名、パス、ディレクトリについて理解する必要があります。

## ファイル名とパス

ファイルは**ディレクトリ**（フォルダとも呼ばれます）に整理されています。
実行中のプログラムには**現在の作業ディレクトリ**があり、ほとんどの操作のデフォルトディレクトリとして機能します。
たとえば、ファイルを開くとき、Pythonは現在の作業ディレクトリ内でファイルを探します。

`os`モジュールは、ファイルやディレクトリを操作するための機能を提供します（「os」は「オペレーティングシステム」を意味します）。
このモジュールには、現在の作業ディレクトリの名前を取得する`getcwd`という関数が用意されています。

In [ ]:
# this cell replaces `os.cwd` with a function that returns a fake path

import os

def getcwd():
    return "/home/dinsdale"

os.getcwd = getcwd

In [ ]:
import os

os.getcwd()

この例の結果は、`dinsdale` という名前のユーザーのホームディレクトリです。`'/home/dinsdale'`のようにファイルやディレクトリを識別する文字列は、**パス**と呼ばれます。

単純なファイル名である `'memo.txt'` もパスと見なされますが、これは現在のディレクトリに対する相対的なファイル名を指定しているため、**相対パス**と呼ばれます。この例では、現在のディレクトリは `/home/dinsdale` なので、`'memo.txt'` は完全なパス `'/home/dinsdale/memo.txt'` と同等です。

`/` で始まるパスは、現在のディレクトリに依存しないため、**絶対パス**と呼ばれます。ファイルへの絶対パスを見つけるためには、`abspath` を使用できます。

In [ ]:
os.path.abspath('memo.txt')

`os`モジュールは、ファイル名やパスを扱うための他の機能も提供しています。
`listdir`は、指定したディレクトリの内容をリストとして返し、ファイルや他のディレクトリも含まれます。
以下は、`photos`という名前のディレクトリの内容を列挙する例です。

In [ ]:
os.listdir('photos')

このディレクトリには、`notes.txt`という名前のテキストファイルと3つのディレクトリが含まれています。ディレクトリにはJPEG形式の画像ファイルが含まれています。

In [ ]:
os.listdir('photos/jan-2023')

ファイルやディレクトリが存在するかどうかを確認するには、`os.path.exists`を使用できます。

In [ ]:
os.path.exists('photos')

In [ ]:
os.path.exists('photos/apr-2023')

パスがファイルかディレクトリかを確認するには、`isdir`を使用できます。この関数は、パスがディレクトリを指している場合に`True`を返します。

In [ ]:
os.path.isdir('photos')

そして、`isfile` はパスがファイルを指している場合に `True` を返します。

In [ ]:
os.path.isfile('photos/notes.txt')

パスを扱う際の課題の一つは、異なるオペレーティングシステムで見た目が異なることです。  
macOSやLinuxのようなUNIXシステムでは、パス内のディレクトリやファイル名はスラッシュ（`/`）で区切られます。  
Windowsではバックスラッシュ（`\`）が使用されます。  
したがって、これらの例をWindows上で実行すると、パスにバックスラッシュが表示され、例の中のスラッシュを置き換える必要があります。

また、両方のシステムで動作するコードを書くためには、`os.path.join`を使用できます。これを使うと、使用するオペレーティングシステムに応じてスラッシュまたはバックスラッシュを使用して、ディレクトリとファイル名を結合してパスを作成できます。

In [ ]:
os.path.join('photos', 'jan-2023', 'photo1.jpg')

この章の後半では、これらの関数を使用してディレクトリのセットを検索し、すべての画像ファイルを見つけます。

## f-strings

プログラムがデータを保存する方法の一つとして、テキストファイルに書き込むというものがあります。
例えば、あなたがラクダ観察者で、観察期間中に見たラクダの数を記録したいとします。
1年半の間に23頭のラクダを見つけたとしましょう。
あなたのラクダ観察ノートのデータはこのようになるかもしれません。

In [ ]:
num_years = 1.5
num_camels = 23

このデータをファイルに書き込むには、第8章で見たように、`write`メソッドを使用できます。`write`の引数は文字列である必要があるので、他の値をファイルに入れたい場合は、それらを文字列に変換する必要があります。これを行う最も簡単な方法は、組み込み関数`str`を使用することです。

以下にその例を示します。

In [ ]:
writer = open('camel-spotting-book.txt', 'w')
writer.write(str(num_years))
writer.write(str(num_camels))
writer.close()

それは動作しますが、`write`は明示的に指定しない限りスペースや改行を追加しません。
ファイルを読み返すと、2つの数字が連続しているのがわかります。

In [ ]:
open('camel-spotting-book.txt').read()

少なくとも、数字の間にスペースを追加するべきです。
ついでに、いくつかの説明文も追加しましょう。

文字列と他の値の組み合わせを書くためには、**f-string**を使用できます。これは、開く引用符の前に `f` という文字があり、中括弧内に1つ以上のPython式を含む文字列です。
以下のf-stringは1つの式を含んでおり、これは変数名です。

In [ ]:
f'I have spotted {num_camels} camels'

結果は、式が評価され、その結果に置き換えられた文字列です。式は複数含まれる場合があります。

In [ ]:
f'In {num_years} years I have spotted {num_camels} camels'

そして、式は演算子や関数呼び出しを含むことができます。

In [ ]:
line = f'In {round(num_years * 12)} months I have spotted {num_camels} camels'
line

このようにデータをテキストファイルに書き込むことができます。

In [ ]:
writer = open('camel-spotting-book.txt', 'w')
writer.write(f'Years of observation: {num_years}\n')
writer.write(f'Camels spotted: {num_camels}\n')
writer.close()

両方のf文字列は、`\n`のシーケンスで終わるため、改行文字が追加されます。

ファイルを次のように読み戻すことができます。

In [ ]:
data = open('camel-spotting-book.txt').read()
print(data)

f文字列では、中括弧内の式が文字列に変換されるため、リストや辞書、その他の型を含めることができます。

In [ ]:
t = [1, 2, 3]
d = {'one': 1}
f'Here is a list {t} and a dictionary {d}'

f文字列に無効な式が含まれている場合、エラーが発生します。

In [ ]:
%%expect TypeError

f'This is not a valid expression {t + 2}'

プログラムがファイルを読み書きする理由の一つに、**設定データ**の保存があります。これは、プログラムが何をすべきか、どのように動作するべきかを指定する情報です。

例えば、重複する写真を検索するプログラムでは、検索すべきディレクトリ名、結果を保存するべき別のディレクトリ名、画像ファイルを識別するために使用するファイル拡張子のリストを含む`config`という辞書を持っているかもしれません。

それは次のように見えるかもしれません：

In [ ]:
config = {
    'photo_dir': 'photos',
    'data_dir': 'photo_info',
    'extensions': ['jpg', 'jpeg'],
}

このデータをテキストファイルに書き込むには、前のセクションで示したようにf文字列を使用することもできますが、このような目的に特化した`yaml`というモジュールを使う方が簡単です。

`yaml`モジュールは、YAMLファイルを扱うための関数を提供します。YAMLファイルは、人間やプログラムが読み書きしやすいようにフォーマットされたテキストファイルです。

以下は、`config`辞書をYAMLファイルに書き込むために`dump`関数を使用する例です。

In [ ]:
# this cell installs the pyyaml package, which provides the yaml module

try:
    import yaml
except ImportError:
    !pip install pyyaml

In [ ]:
import yaml

config_filename = 'config.yaml'
writer = open(config_filename, 'w')
yaml.dump(config, writer)
writer.close()

ファイルの内容を読み返すと、YAML形式がどのようなものかを見ることができます。

In [ ]:
readback = open(config_filename).read()
print(readback)

これで、`safe_load` を使用して YAML ファイルを読み込むことができます。

In [ ]:
reader = open(config_filename)
config_readback = yaml.safe_load(reader)
config_readback

結果として、新しい辞書が作成され、それは元の辞書と同じ情報を含んでいますが、同じ辞書ではありません。

In [ ]:
config is config_readback

辞書のようなオブジェクトを文字列に変換することを**シリアライズ**と呼びます。  
文字列を再びオブジェクトに変換することを**デシリアライズ**と呼びます。  
オブジェクトをシリアライズしてデシリアライズすると、その結果は元のオブジェクトと同等であるべきです。

## Shelve

これまで、テキストファイルの読み書きを行ってきましたが、ここでデータベースを考えてみましょう。
**データベース**とは、データを保存するために組織化されたファイルのことです。
データベースの中には、情報の行と列があるテーブルのように組織化されたものもあれば、キーから値へのマッピングのように組織化された辞書のようなものもあります。これらは**キー・バリュー・ストア**と呼ばれることもあります。

`shelve`モジュールは、「シェルフ」と呼ばれるキー・バリュー・ストアを作成および更新するための関数を提供しています。
例として、`photos`ディレクトリ内の図のキャプションを含むシェルフを作成します。
シェルフを配置するディレクトリの名前を取得するために、`config`辞書を使用します。

In [ ]:
config['data_dir']

このディレクトリがまだ存在しない場合、`os.makedirs`を使用して作成することができます。

In [ ]:
os.makedirs(config['data_dir'], exist_ok=True)

そして、ディレクトリ名とシェルフファイル名「captions」を含むパスを作成するために、`os.path.join`を使用します。

In [ ]:
db_file = os.path.join(config['data_dir'], 'captions')
db_file

これで、`shelve.open`を使用してシェルフファイルを開くことができます。引数`c`は、必要に応じてファイルを作成することを示しています。

In [ ]:
import shelve

db = shelve.open(db_file, 'c')
db

戻り値は公式には `DbfilenameShelf` オブジェクトであり、よりカジュアルにはシェルフオブジェクトと呼ばれます。

シェルフオブジェクトは多くの点で辞書のように振る舞います。例えば、ブラケット演算子を使用してアイテムを追加できます。これはキーと値のマッピングです。

In [ ]:
key = 'jan-2023/photo1.jpg'
db[key] = 'Cat nose'

この例では、キーが画像ファイルへのパスであり、値が画像を説明する文字列です。

また、ブラケット演算子を使用してキーを検索し、対応する値を取得します。

In [ ]:
value = db[key]
value

既存のキーに別の値を割り当てると、`shelve`は古い値を置き換えます。

In [ ]:
db[key] = 'Close up view of a cat nose'
db[key]

`keys`、`values`、`items`のような一部の辞書メソッドは、shelfオブジェクトでも機能します。

In [ ]:
list(db.keys())

In [ ]:
list(db.values())

`in` 演算子を使用して、棚にキーが存在するかどうかを確認できます。

In [ ]:
key in db

`for`文を使用してキーをループ処理することができます。

In [ ]:
for key in db:
    print(key, ':', db[key])

他のファイルと同様に、作業が終わったらデータベースを閉じるべきです。

In [ ]:
db.close()

データディレクトリの内容を一覧表示すると、2つのファイルが見えます。

In [ ]:
# When you open a shelve file, a backup file is created that has the suffix `.bak`.
# If you run this notebook more than once, you might see that file left behind.
# This cell removes it so the output shown in the book is correct.

!rm -f photo_info/captions.bak

In [ ]:
os.listdir(config['data_dir'])

`captions.dat`には、私たちがちょうど保存したデータが含まれています。
`captions.dir`には、データベースの組織に関する情報が含まれており、それによってアクセスがより効率的になります。
`dir`というサフィックスは「ディレクトリ」を表していますが、ファイルを含むディレクトリとは関係ありません。

## データ構造の保存

前の例では、shelfのキーと値は文字列でしたが、リストや辞書のようなデータ構造を含めるためにもshelfを使用することができます。

例として、第11章の演習で扱ったアナグラムの例を再訪しましょう。覚えているかもしれませんが、私たちはアルファベット順に並べた文字列から、その文字でつづることができる単語のリストにマッピングする辞書を作成しました。
例えば、キー `'opst'` はリスト `['opts', 'post', 'pots', 'spot', 'stop', 'tops']` にマッピングされます。

以下の関数を使用して、単語の文字を並べ替えます。

In [ ]:
def sort_word(word):
    return ''.join(sorted(word))

そしてこちらが例です。

In [ ]:
word = 'pots'
key = sort_word(word)
key

では、「anagram_map」という名前の棚を開いてみましょう。引数 `'n'` は、新しい空の棚を常に作成することを意味します。既に存在している場合でも同様です。

In [ ]:
db = shelve.open('anagram_map', 'n')

これで、棚にアイテムを追加することができます。

In [ ]:
db[key] = [word]
db[key]

この項目では、キーは文字列で、値は文字列のリストです。

さて、例えば「tops」のように同じ文字を含む別の単語を見つけたとします。

In [ ]:
word = 'tops'
key = sort_word(word)
key

キーは前の例と同じなので、同じ文字列のリストに2番目の単語を追加したいと思います。
もし `db` が辞書である場合、私たちは次のようにそれを行います。

In [ ]:
db[key].append(word)          # INCORRECT

しかし、それを実行してからシェルフでキーを調べると、更新されていないように見えます。

In [ ]:
db[key]

問題があります。キーを調べると文字列のリストが得られますが、文字列のリストを変更してもシェルフには影響しません。シェルフを更新したい場合は、古い値を読み取り、それを更新して、新しい値をシェルフに書き戻す必要があります。

In [ ]:
anagram_list = db[key]
anagram_list.append(word)
db[key] = anagram_list

棚の値が更新されました。

In [ ]:
db[key]

練習として、この例を完了するには、単語リストを読み取り、すべてのアナグラムをシェルフに保存することができます。

In [ ]:
db.close()

## 等価なファイルのチェック

さて、この章の目的に戻りましょう。異なるファイルで同じデータを含むものを検索することです。
その1つの方法は、両方のファイルの内容を読み込んで比較することです。

ファイルが画像を含む場合、モードを `'rb'` にして開く必要があります。ここで `'r'` は内容を読みたいことを意味し、`'b'` は**バイナリモード**を示します。
バイナリモードでは、内容はテキストとして解釈されず、バイトのシーケンスとして扱われます。

以下は画像ファイルを開いて読み込む例です。

In [ ]:
path1 = 'photos/jan-2023/photo1.jpg'
data1 = open(path1, 'rb').read()
type(data1)

`read` の結果は `bytes` オブジェクトです。その名の通り、バイト列を含んでいます。

一般的に、画像ファイルの内容は人間には読みやすいものではありません。しかし、別のファイルからその内容を読み取れば、`==` 演算子を使用して比較することができます。

In [ ]:
path2 = 'photos/jan-2023/photo2.jpg'
data2 = open(path2, 'rb').read()
data1 == data2

これらの2つのファイルは同等ではありません。

これまでの内容を関数にまとめましょう。

In [ ]:
def same_contents(path1, path2):
    data1 = open(path1, 'rb').read()
    data2 = open(path2, 'rb').read()
    return data1 == data2

ファイルが2つだけの場合、この関数は良い選択肢です。しかし、膨大な数のファイルがあり、それらのうちのいずれか2つが同じデータを含んでいるかどうかを知りたいと仮定します。すべてのファイルペアを比較するのは非効率的でしょう。

代替方法として**ハッシュ関数**を使用できます。ハッシュ関数は、ファイルの内容を取り込み、通常は大きな整数である**ダイジェスト**を生成します。2つのファイルが同じデータを含んでいる場合、同じダイジェストを持つことになります。2つのファイルが異なる場合、*ほとんど常に*異なるダイジェストを持ちます。

`hashlib`モジュールは、いくつかのハッシュ関数を提供します。ここで使用するのは`md5`というものです。まず、`hashlib.md5`を使用して`HASH`オブジェクトを作成してみましょう。

In [ ]:
import hashlib

md5_hash = hashlib.md5()
type(md5_hash)

`HASH`オブジェクトは、ファイルの内容を引数として受け取る`update`メソッドを提供します。

In [ ]:
md5_hash.update(data1)

これで `hexdigest` を使用して、16進数で整数を表す文字列としてダイジェストを取得できます。

In [ ]:
digest = md5_hash.hexdigest()
digest

次の関数はこれらのステップをカプセル化します。

In [ ]:
def md5_digest(filename):
    data = open(filename, 'rb').read()
    md5_hash = hashlib.md5()
    md5_hash.update(data)
    digest = md5_hash.hexdigest()
    return digest

異なるファイルの内容をハッシュ化すると、異なるダイジェストが得られることを確認できます。

In [ ]:
filename2 = 'photos/feb-2023/photo2.jpg'
md5_digest(filename2)

これで、同等のファイルを見つけるために必要なものがほぼすべて揃いました。最後のステップは、ディレクトリを検索してすべての画像ファイルを見つけることです。

## ディレクトリの探索

以下の関数は、検索したいディレクトリを引数として受け取ります。
`listdir` を使ってディレクトリの内容をループ処理します。
ファイルを見つけた場合、その完全なパスを表示します。
ディレクトリを見つけた場合、再帰的に自身を呼び出してサブディレクトリを探索します。

In [ ]:
def walk(dirname):
    for name in os.listdir(dirname):
        path = os.path.join(dirname, name)

        if os.path.isfile(path):
            print(path)
        elif os.path.isdir(path):
            walk(path)

このように使用できます。

In [ ]:
walk('photos')

結果の順序はオペレーティングシステムの詳細によって異なります。

## デバッグ

ファイルを読み書きしていると、空白文字が原因で問題が発生することがあります。
空白文字は通常目に見えないため、これらのエラーのデバッグは難しい場合があります。
たとえば、次に示す文字列には空白、タブ（シーケンス `\t` で表される）、および改行（シーケンス `\n` で表される）が含まれています。
これをプリントするときに、空白文字は見えません。

In [ ]:
s = '1 2\t 3\n 4'
print(s)

組み込み関数 `repr` は役に立ちます。この関数は任意のオブジェクトを引数として取ることができ、そのオブジェクトの文字列表現を返します。文字列に対しては、空白文字をバックスラッシュシーケンスで表現します。

In [ ]:
print(repr(s))

これはデバッグに役立つことがあります。

もう一つの問題として、異なるシステムが行の終わりを示すために異なる文字を使用することがあります。あるシステムでは、改行を使用し、これは`\n`で表されます。他のシステムでは、復帰文字を使用し、これは`\r`で表されます。また、両方を使用するシステムもあります。異なるシステム間でファイルを移動するとき、これらの不一致が問題を引き起こすことがあります。

ファイル名の大文字小文字の扱いも、異なるオペレーティングシステムを使用する際に直面する可能性のある問題です。macOSやUNIXでは、ファイル名には小文字と大文字、数字、そしてほとんどの記号を含めることができます。しかし、Windowsの多くのアプリケーションでは小文字と大文字の違いを無視し、macOSやUNIXで許可されているいくつかの記号がWindowsでは許可されていません。

了解しました。こちらの用語集には、プログラミングに関連する英単語とその意味が記載されています。それぞれの用語は日本語で次のように説明できます。

**ephemeral（短命な）:**
短期間だけ実行され、終了するとそのデータが失われるプログラム。

**persistent（永続的な）:**
無期限に実行され、少なくとも一部のデータを恒久的なストレージに保存するプログラム。

**directory（ディレクトリ）:**
ファイルや他のディレクトリの集合。

**current working directory（現在の作業ディレクトリ）:**
プログラムが別のディレクトリを指定しない限り、デフォルトで使用されるディレクトリ。

**path（パス）:**
通常はファイルに至るディレクトリの順序を指定する文字列。

**relative path（相対パス）:**
カレントディレクトリまたは他の指定されたディレクトリから始まるパス。

**absolute path（絶対パス）:**
現在のディレクトリに依存しないパス。

**f-string（f文字列）:**
開始引用符の前に「f」が付き、中括弧で囲まれた1つ以上の式を含む文字列。

**configuration data（設定データ）:**
プログラムの動作や方法を指定する、しばしばファイルに保存されるデータ。

**serialization（シリアライズ）:**
オブジェクトを文字列に変換すること。

**deserialization（デシリアライズ）:**
文字列をオブジェクトに変換すること。

**database（データベース）:**
特定の操作を効率的に行うために内容が組織化されたファイル。

**key-value stores（キー値ストア）:**
キーが値に対応する辞書のように内容が組織されたデータベース。

**binary mode（バイナリモード）:**
ファイルの内容を文字の並びとしてではなくバイトの並びとして解釈する方法でファイルを開くこと。

**hash function（ハッシュ関数）:**
オブジェクトを受け取り整数を計算する関数で、時にはダイジェストと呼ばれることがある。

**digest（ダイジェスト）:**
特に2つのオブジェクトが同じかどうかを確認するために用いるハッシュ関数の結果。

## エクササイズ

In [ ]:
# This cell tells Jupyter to provide detailed debugging information
# when a runtime error occurs. Run it before working on the exercises.

%xmode Verbose

### バーチャルアシスタントに尋ねる

この章で触れた話題にはいくつかの詳細を説明していないものがあります。
バーチャルアシスタントに詳細情報を求めるために、次のような質問をしてみてください。

* 「揮発性プログラムと永続性プログラムの違いは何ですか？」

* 「永続性プログラムの例としてはどのようなものがありますか？」

* 「相対パスと絶対パスの違いは何ですか？」

* 「なぜ `yaml` モジュールには `load` と `safe_load` という関数があるのですか？」

* 「Pythonのshelfを書くとき、`dat`と`dir`というサフィックスがついたファイルは何ですか？」

* 「キー・バリュー・ストア以外に、どのような種類のデータベースがありますか？」

* 「ファイルを読むとき、バイナリモードとテキストモードの違いは何ですか？」

* 「バイトオブジェクトと文字列の違いは何ですか？」

* 「ハッシュ関数とは何ですか？」

* 「MD5ダイジェストとは何ですか？」

いつも通り、以下の演習で行き詰まった場合は、VAに助けを求めることを考慮してください。質問と一緒に、この章の関連機能を貼り付けると良いでしょう。

### 演習

`replace_all`という関数を書いてください。この関数はパターン文字列、置換文字列、そして2つのファイル名を引数として受け取ります。
この関数は最初のファイルを読み込み、その内容を2番目のファイルに書き込みます（必要ならばファイルを作成します）。
内容の中にパターン文字列がどこかに現れた場合、それを置換文字列に置き換えなければなりません。

以下は、開始するための関数のアウトラインです。

In [ ]:
def replace_all(old, new, source_path, dest_path):
    # read the contents of the source file
    reader = open(source_path)

    # replace the old string with the new

    # write the result into the destination file


In [ ]:
# Solution goes here

申し訳ありませんが、ファイルを直接操作する能力はありません。しかし、ファイル操作はPythonを使って簡単に行えます。以下はそのためのサンプルコードです。

```python
# ファイル 'photos/notes.txt' を開く
with open('photos/notes.txt', 'r') as file:
    # ファイル内容を読み込む
    content = file.read()

# 'photos' という単語を 'images' に置換
new_content = content.replace('photos', 'images')

# 変更した内容を新たなファイル 'photos/new_notes.txt' に書き込む
with open('photos/new_notes.txt', 'w') as new_file:
    new_file.write(new_content)
```

このコードを実行することで、指定された操作を行うことができます。何か他にお手伝いできることがあればお知らせください。

In [ ]:
source_path = 'photos/notes.txt'
open(source_path).read()

In [ ]:
dest_path = 'photos/new_notes.txt'
old = 'photos'
new = 'images'
replace_all(old, new, source_path, dest_path)

In [ ]:
open(dest_path).read()

### 演習

[前のセクション](section_storing_data_structure)では、`shelve`モジュールを使用して、ソートされた文字列とアナグラムのリストとの間のキーバリューストアを作成しました。  
この例を完成させるために、文字列とshelveオブジェクトを引数に取る関数`add_word`を作成してください。

この関数は、単語の文字をソートしてキーを作成し、キーがすでにshelfに存在するかを確認します。存在しない場合は、新しい単語を含むリストを作成し、それをshelfに追加します。もし存在する場合は、新しい単語を既存の値に追加します。

In [ ]:
# Solution goes here

このループを使用して、関数をテストすることができます。

In [ ]:
download('https://raw.githubusercontent.com/AllenDowney/ThinkPython/v3/words.txt');

In [ ]:
word_list = open('words.txt').read().split()

db = shelve.open('anagram_map', 'n')
for word in word_list:
    add_word(word, db)

すべてが正常に動作していれば、`'opst'`のようなキーを調べて、その文字で綴られる単語のリストを取得できるはずです。

In [ ]:
db['opst']

In [ ]:
for key, value in db.items():
    if len(value) > 8:
        print(value)

In [ ]:
db.close()

### エクササイズ

大量のファイルコレクションの中で、異なるディレクトリに保存されたり異なるファイル名で保存されたりしている同じファイルのコピーが複数あるかもしれません。
このエクササイズの目的は、重複ファイルを検索することです。
例として、`photos` ディレクトリ内の画像ファイルを使用します。

次のように動作します：

* `walk` 関数を使用して、`config['extensions']` に含まれる拡張子で終わるファイルをこのディレクトリで探します。

* 各ファイルに対して、`md5_digest` を使用して内容のダイジェストを計算します。

* シェルフを使用して、各ダイジェストからそのダイジェストを持つパスのリストへのマッピングを作成します。

* 最後に、複数のファイルにマップされているダイジェストがないかシェルフを検索します。

* もし見つかった場合、`same_contents` を使用して、ファイルが同一のデータを含むかを確認します。

まず最初にいくつかの関数を書くことを提案します。それから全てをまとめます。

1. 画像ファイルを識別するために、`is_image`という関数を書きます。この関数はパスとファイル拡張子のリストを引数として受け取り、パスがリスト内の一つの拡張子で終了する場合に`True`を返します。ヒント: `os.path.splitext`を使用します。

以下はその関数の例です：

```python
import os

def is_image(path, extensions):
    _, ext = os.path.splitext(path)
    return ext.lower() in (e.lower() for e in extensions)
```

この関数は、指定されたパスが画像ファイルの拡張子リストに含まれる場合に`True`を返します。

In [ ]:
# Solution goes here

`doctest` を使用して関数をテストすることができます。

In [ ]:
from doctest import run_docstring_examples

def run_doctests(func):
    run_docstring_examples(func, globals(), name=func.__name__)

run_doctests(is_image)

```python
import hashlib
import shelve

def md5_digest(file_path):
    """Returns the MD5 digest of the contents of the file at `file_path`."""
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

def add_path(path, shelf):
    """Adds the path to the shelf, keyed by the file's MD5 digest."""
    # Compute the MD5 digest of the file
    digest = md5_digest(path)
    
    # Open the shelf and update it
    with shelve.open(shelf, writeback=True) as db:
        if digest not in db:
            db[digest] = [path]
        else:
            # Avoid adding duplicate paths
            if path not in db[digest]:
                db[digest].append(path)
```

このコードは、`md5_digest` 関数を用いてファイルのMD5ダイジェストを計算し、そのダイジェストをキーとしてパスを棚（shelf）に追加します。棚には、ダイジェストがすでに存在する場合、そのリストに新しいパスを追加し、存在しない場合は新しいリストを作成します。

In [ ]:
# Solution goes here

ディレクトリとそのサブディレクトリ内のファイルを走査する `walk_images` というバージョンの `walk` を作成します。各ファイルについて、`is_image` を使用してそれが画像ファイルかどうかを確認し、`add_path` を使用して棚に追加する必要があります。以下はそのPythonコードの例です。

```python
import os

def is_image(file_path):
    # 画像ファイルの拡張子を定義
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff']
    _, ext = os.path.splitext(file_path)
    return ext.lower() in image_extensions

def add_path(shelf, file_path):
    # ファイルパスを棚に追加する処理
    shelf.append(file_path)

def walk_images(directory, shelf):
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            if is_image(file_path):
                add_path(shelf, file_path)

# 使用例:
# ディレクトリパスを指定
directory_path = '/path/to/directory'
shelf = []

# walk_images を呼び出して画像ファイルを収集
walk_images(directory_path, shelf)

# 結果を表示
print(shelf)
```

このコードでは、指定されたディレクトリ内のすべてのファイルについて、`is_image` 関数を使って画像ファイルかどうかを確認し、画像ファイルであれば `add_path` 関数で棚にそのパスを追加します。`os.walk` を使用することで、サブディレクトリも含めて走査が可能です。

In [ ]:
# Solution goes here

すべてが正常に機能している場合、次のプログラムを使用してシェルフを作成し、`photos` ディレクトリを検索してシェルフにパスを追加し、同じダイジェストを持つファイルが複数存在するかどうかを確認できます。

In [ ]:
db = shelve.open('photos/digests', 'n')
walk_images('photos')

for digest, paths in db.items():
    if len(paths) > 1:
        print(paths)

同じダイジェストを持つファイルのペアを見つける必要があります。`same_contents`を使用して、それらが同じデータを含んでいるかどうかを確認してください。

In [ ]:
# Solution goes here

[Think Python: 第3版](https://allendowney.github.io/ThinkPython/index.html)

Copyright 2024 [Allen B. Downey](https://allendowney.com)

コードライセンス: [MITライセンス](https://mit-license.org/)

テキストライセンス: [クリエイティブ・コモンズ 表示-非営利-継承4.0国際](https://creativecommons.org/licenses/by-nc-sa/4.0/)